# *Lending Club Training*

In this tutorial, we will go through a step-by-step workflow to determine loan deliquency.  Predictions are made based on the information available at the time the loan was issued.  Our data is a portion of the public Lending Club dataset.

## Workflow

1. Start H2O-3 cluster
2. Import data
3. Clean data
4. Feature engineering
5. Model training
6. Examine model accuracy
7. Interpret model
8. Stop H2O-3 cluster

# Step 1 (of 8). Start H2O-3 cluster

In [92]:
import h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_181"; Java(TM) SE Runtime Environment (build 1.8.0_181-b13); Java HotSpot(TM) 64-Bit Server VM (build 25.181-b13, mixed mode)
  Starting server from /Users/sshkel/miniconda3/envs/h2o/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/gw/7xx4qg7j3ns6bcjl6p8pr_nszhzqmk/T/tmpjeulswy0
  JVM stdout: /var/folders/gw/7xx4qg7j3ns6bcjl6p8pr_nszhzqmk/T/tmpjeulswy0/h2o_sshkel_started_from_python.out
  JVM stderr: /var/folders/gw/7xx4qg7j3ns6bcjl6p8pr_nszhzqmk/T/tmpjeulswy0/h2o_sshkel_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,01 secs
H2O cluster timezone:,Pacific/Auckland
H2O data parsing timezone:,UTC
H2O cluster version:,3.20.0.9
H2O cluster version age:,23 days
H2O cluster name:,H2O_from_python_sshkel_zz47h2
H2O cluster total nodes:,1
H2O cluster free memory:,3.556 Gb
H2O cluster total cores:,12
H2O cluster allowed cores:,12
H2O cluster status:,"accepting new members, healthy"


# Step 2 (of 8). Import data

### View Data

In [93]:
data_path ="https://s3.amazonaws.com/h2o-public-test-data/bigdata/laptop/lending-club/LoanStats3a.csv"
loans = h2o.import_file(data_path,
                        col_types = {"int_rate":"string", 
                                     "revol_util":"string", 
                                     "emp_length":"string", 
                                     "verification_status":"string"})

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [94]:
loans.dim

[42538, 52]

In [95]:
loans.head()

id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code
1.0775e+06,1.2966e+06,5000,5000,4975,36 months,10.65%,162.87,B,B2,,10+ years,RENT,24000,VERIFIED - income,2011-12-01 00:00:00,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail.action?loan_id=1077501,Borrower added on 12/22/11 > I need to upgrade my business technologies.,credit_card,Computer,860xx,AZ,27.65,0,1985-01-01 00:00:00,1,,,3,0,13648,83.7%,9,f,0,0,5861.07,5831.78,5000,861.07,0,0,0,2015-01-01 00:00:00,171.62,,2015-01-01 00:00:00,0,,1
1.07743e+06,1.31417e+06,2500,2500,2500,60 months,15.27%,59.83,C,C4,Ryder,< 1 year,RENT,30000,VERIFIED - income source,2011-12-01 00:00:00,Charged Off,n,https://www.lendingclub.com/browse/loanDetail.action?loan_id=1077430,Borrower added on 12/22/11 > I plan to use this money to finance the motorcycle i am looking at. I plan to have it paid off as soon as possible/when i sell my old bike. I only need this money because the deal im looking at is to good to pass up. Borrower added on 12/22/11 > I plan to use this money to finance the motorcycle i am looking at. I plan to have it paid off as soon as possible/when i sell my old bike.I only need this money because the deal im looking at is to good to pass up. I have finished college with an associates degree in business and its takingmeplaces,car,bike,309xx,GA,1,0,1999-04-01 00:00:00,5,,,3,0,1687,9.4%,4,f,0,0,1008.71,1008.71,456.46,435.17,0,117.08,1.11,2013-04-01 00:00:00,119.66,,2013-09-01 00:00:00,0,,1
1.07718e+06,1.31352e+06,2400,2400,2400,36 months,15.96%,84.33,C,C5,,10+ years,RENT,12252,not verified,2011-12-01 00:00:00,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail.action?loan_id=1077175,,small_business,real estate business,606xx,IL,8.72,0,2001-11-01 00:00:00,2,,,2,0,2956,98.5%,10,f,0,0,3003.65,3003.65,2400,603.65,0,0,0,2014-06-01 00:00:00,649.91,,2015-06-01 00:00:00,0,,1
1.07686e+06,1.27718e+06,10000,10000,10000,36 months,13.49%,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200,VERIFIED - income source,2011-12-01 00:00:00,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail.action?loan_id=1076863,"Borrower added on 12/21/11 > to pay for property tax (borrow from friend, need to pay back) & central A/C need to be replace. I'm very sorry to let my loan expired last time.",other,personel,917xx,CA,20,0,1996-02-01 00:00:00,1,35,,10,0,5598,21%,37,f,0,0,12226.3,12226.3,10000,2209.33,16.97,0,0,2015-01-01 00:00:00,357.48,,2015-01-01 00:00:00,0,,1
1.07536e+06,1.31175e+06,3000,3000,3000,60 months,12.69%,67.79,B,B5,University Medical Group,1 year,RENT,80000,VERIFIED - income source,2011-12-01 00:00:00,Current,n,https://www.lendingclub.com/browse/loanDetail.action?loan_id=1075358,"Borrower added on 12/21/11 > I plan on combining three large interest bills together and freeing up some extra each month to pay toward other bills. I've always been a good payor but have found myself needing to make adjustments to my budget due to a medical scare. My job is very stable, I love it.",other,Personal,972xx,OR,17.94,0,1996-01-01 00:00:00,0,38,,15,0,27783,53.9%,38,f,1168.04,1168.04,2767.64,2767.64,1831.96,935.68,0,0,0,2015-06-01 00:00:00,67.79,2015-07-01 00:00:00,2015-06-01 00:00:00,0,,1
1.07527e+06,1.31144e+06,5000,5000,5000,36 months,7.90%,156.46,A,A4,Veolia Transportaton,3 years,RENT,36000,VERIFIED - income source,2011-12-01 00:00:00,Fully Paid,n,https://www.lendingclub.com/browse/loanD

### Filter Loans

Now we will filter out loans that are ongoing.  These are loans with loan status like "Current" and "In Grace Period".

In [96]:
num_unfiltered_loans = loans.dim[0]
num_unfiltered_loans

42538

In [97]:
loans["loan_status"].table().head(20)

loan_status,Count
Charged Off,5435
Current,3351
Default,7
Does not meet the credit policy. Status:Charged Off,761
Does not meet the credit policy. Status:Current,53
Does not meet the credit policy. Status:Fully Paid,1933
Does not meet the credit policy. Status:In Grace Period,2
Fully Paid,30843
In Grace Period,60
Late (16-30 days),16


In [98]:
ongoing_status = ["Current",
                  "In Grace Period",
                  "Late (16-30 days)",
                  "Late (31-120 days)"]
loans = loans[~loans["loan_status"].isin(ongoing_status)]

In [99]:
num_filtered_loans = loans.dim[0]
num_filtered_loans

39037

In [100]:
num_loans_filtered_out = num_unfiltered_loans - num_filtered_loans
num_loans_filtered_out

3501

### Create Response Column

Our response column will be called: `bad_loan`.  The `bad_loan` column will be positive if the loan was not completely paid off.

In [101]:
loans["bad_loan"] = ~(loans["loan_status"] == "Fully Paid")
loans["bad_loan"] = loans["bad_loan"].asfactor() # convert to enum/factor

In [102]:
bad_loan_dist = loans["bad_loan"].table()
bad_loan_dist["Percentage"] = (100*bad_loan_dist["Count"]/loans.nrow).round()
bad_loan_dist

bad_loan,Count,Percentage
0,30843,79
1,8194,21


About one in five loans eventually become bad.

# Step 3 (of 8).  Clean data

We have multiple columns that are numeric but are being treated as string because of their syntax.  In this section, we will convert these to numeric.  Our machine learning models will have a greater ability to understand numeric features than strings.

In [103]:
loans[["int_rate", "revol_util", "emp_length"]].head()

int_rate,revol_util,emp_length
10.65%,83.7%,10+ years
15.27%,9.4%,< 1 year
15.96%,98.5%,10+ years
13.49%,21%,10+ years
7.90%,28.3%,3 years
18.64%,87.5%,9 years
21.28%,32.6%,4 years
12.69%,36.5%,< 1 year
14.65%,20.6%,5 years
12.69%,67.1%,10+ years


In [104]:
# Convert int_rate to numeric
loans["int_rate"] = loans["int_rate"].gsub(pattern = "%", replacement = "") # strip %
loans["int_rate"] = loans["int_rate"].trim() # trim ws
loans["int_rate"] = loans["int_rate"].asnumeric() #change to a numeric 

In [105]:
loans["int_rate"].head()

int_rate
10.65
15.27
15.96
13.49
7.9
18.64
21.28
12.69
14.65
12.69


Now that we have converted interest rate to numeric, we can use the `hist` function to see the distribution of interest rate for good loans and bad loans.

 The distribution of interest rate is very different for good loans.  This may be a helpful predictor in our model.

In [106]:
# Convert revol_util to numeric
loans["revol_util"] = loans["revol_util"].gsub(pattern = "%", replacement = "") # strip %
loans["revol_util"] = loans["revol_util"].trim() # trim ws
loans["revol_util"] = loans["revol_util"].asnumeric() #change to a numeric 

In [107]:
# Convert emp_length to numeric
# Use gsub to remove " year" and " years" also translate n/a to "" 
loans["emp_length"] = loans["emp_length"].gsub(pattern = "([ ]*+[a-zA-Z].*)|(n/a)", replacement = "") 

# Use trim to remove any trailing spaces 
loans["emp_length"] = loans["emp_length"].trim()

# Convert emp_length to numeric 
# Use sub to convert < 1 to 0 years and do the same for 10 + to 10
# Hint: Be mindful of spaces between characters
loans["emp_length"] = loans["emp_length"].gsub(pattern = "< 1", replacement = "0")
loans["emp_length"] = loans["emp_length"].gsub(pattern = "10\\+", replacement = "10")
loans["emp_length"] = loans["emp_length"].asnumeric()

In [108]:
loans[["int_rate", "revol_util", "emp_length"]].head()

int_rate,revol_util,emp_length
10.65,83.7,10
15.27,9.4,0
15.96,98.5,10
13.49,21,10
7.9,28.3,3
18.64,87.5,9
21.28,32.6,4
12.69,36.5,0
14.65,20.6,5
12.69,67.1,10


We can also clean up the verification status column. There are multiple values that mean verified: `VERIFIED - income` and `VERIFIED - income source`.  We will replace these values with `verified`.

In [109]:
loans["verification_status"].head()

verification_status
VERIFIED - income
VERIFIED - income source
not verified
VERIFIED - income source
VERIFIED - income source
VERIFIED - income source
VERIFIED - income source
VERIFIED - income
not verified
VERIFIED - income source


In [110]:
loans["verification_status"] = loans["verification_status"].sub(pattern = "VERIFIED - income source", 
                                                                replacement = "verified")
loans["verification_status"] = loans["verification_status"].sub(pattern = "VERIFIED - income", 
                                                                replacement = "verified")
loans["verification_status"] = loans["verification_status"].asfactor()

In [111]:
loans["verification_status"].table()

verification_status,Count
not verified,17965
verified,21069


# Step 4 (of 8).  Feature engineering

Now that we have cleaned our data, we can add some new columns to our dataset that may help improve the performance of our supervised learning models.

The new columns we will create are: 
* credit_length: the time from their earliest credit line to when they were issued the loan
* expansion of issue date: extract year and month from the issue date
* word embeddings from the loan description

### Credit Length

We can extract the credit length by subtracting the year they had their earliest credit line from the year when they issued the loan.

In [112]:
loans["credit_length"] = loans["issue_d"].year() - loans["earliest_cr_line"].year()
loans["credit_length"].head()

credit_length
26
12
10
15
7
4
7
7
13
22


### Issue Date Expansion

We can extract the year and month from the issue date.  We may find that the month or the year when the loan was issued can impact the probability of a bad loan.

In [113]:
loans["issue_d_year"] = loans["issue_d"].year()
loans["issue_d_month"] = loans["issue_d"].month().asfactor()  # we will treat month as a enum/factor since its cyclical

In [114]:
loans[["issue_d_year", "issue_d_month"]].head()

issue_d_year,issue_d_month
2011,12
2011,12
2011,12
2011,12
2011,12
2011,12
2011,12
2011,12
2011,12
2011,12


### Word Embeddings

One of the columns in our dataset is a description of why the loan was requested. The first few descriptions in the dataset are shown below.

In [115]:
loans["desc"].head()

desc
Borrower added on 12/22/11 > I need to upgrade my business technologies.
Borrower added on 12/22/11 > I plan to use this money to finance the motorcycle i am looking at. I plan to have it paid off as soon as possible/when i sell my old bike. I only need this money because the deal im looking at is to good to pass up. Borrower added on 12/22/11 > I plan to use this money to finance the motorcycle i am looking at. I plan to have it paid off as soon as possible/when i sell my old bike.I only need this money because the deal im looking at is to good to pass up. I have finished college with an associates degree in business and its takingmeplaces
""
"Borrower added on 12/21/11 > to pay for property tax (borrow from friend, need to pay back) & central A/C need to be replace. I'm very sorry to let my loan expired last time."
""
Borrower added on 12/16/11 > Downpayment for a car.
Borrower added on 12/21/11 > I own a small home-based judgment collection business. I have 5 years experience collecting debts. I am now going from a home office to a small office. I also plan to buy a small debt portfolio (eg. $10K for $1M of debt) My score is not A+ because I own my home and have no mortgage.
"Borrower added on 12/16/11 > I'm trying to build up my credit history. I live with my brother and have no car payment or credit cards. I am in community college and work full time. Im going to use the money to make some repairs around the house and get some maintenance done on my car. Borrower added on 12/20/11 > $1000 down only $4375 to go. Thanks to everyone that invested so far, looking forward to surprising my brother with the fixes around the house."
"Borrower added on 12/15/11 > I had recived a loan from Citi Financial about a year ago, I was paying 29.99 intrest, so the refinance is to cut that rate since cleaning up my credit I have been paying everything on time as shown on my credit report"
""


This information may be important to the model but supervised learning algorithms have a hard time understanding text.  Instead we will convert these strings to a numeric vector using the Word2Vec algorithm.

In [116]:
STOP_WORDS = ["ax","i","you","edu","s","t","m","subject","can","lines","re","what",
              "there","all","we","one","the","a","an","of","or","in","for","by","on",
              "but","is","in","a","not","with","as","was","if","they","are","this","and","it","have",
              "from","at","my","be","by","not","that","to","from","com","org","like","likes","so"]

In [117]:
def tokenize(sentences, stop_word = STOP_WORDS):
    tokenized = sentences.tokenize("\\W+")
    tokenized_lower = tokenized.tolower()
    tokenized_filtered = tokenized_lower[(tokenized_lower.nchar() >= 2) | (tokenized_lower.isna()),:]
    tokenized_words = tokenized_filtered[tokenized_filtered.grep("[0-9]",invert=True,output_logical=True),:]
    tokenized_words = tokenized_words[(tokenized_words.isna()) | (~ tokenized_words.isin(STOP_WORDS)),:]
    return tokenized_words

In [118]:
# Break loan description into sequence of words
words = tokenize(loans["desc"].ascharacter())

In [119]:
# Train Word2Vec Model
from h2o.estimators.word2vec import H2OWord2vecEstimator

w2v_model = H2OWord2vecEstimator(vec_size = 100, model_id = "w2v.hex")
w2v_model.train(training_frame=words)

word2vec Model Build progress: |██████████████████████████████████████████| 100%


In [120]:
# Sanity check - find synonyms for the word 'car'
w2v_model.find_synonyms("car", count = 5)

OrderedDict([('auto', 0.7504355907440186),
             ('vehicle', 0.728424072265625),
             ('prius', 0.7114761471748352),
             ('camry', 0.6935193538665771),
             ('acura', 0.6830691695213318)])

In [121]:
# Calculate a vector for each description
desc_vecs = w2v_model.transform(words, aggregate_method = "AVERAGE")

In [122]:
desc_vecs.head()

C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26,C27,C28,C29,C30,C31,C32,C33,C34,C35,C36,C37,C38,C39,C40,C41,C42,C43,C44,C45,C46,C47,C48,C49,C50,C51,C52,C53,C54,C55,C56,C57,C58,C59,C60,C61,C62,C63,C64,C65,C66,C67,C68,C69,C70,C71,C72,C73,C74,C75,C76,C77,C78,C79,C80,C81,C82,C83,C84,C85,C86,C87,C88,C89,C90,C91,C92,C93,C94,C95,C96,C97,C98,C99,C100
-0.0404573,-0.108231,-0.0601569,-0.199214,0.103919,-0.0695549,0.0695383,0.0602171,0.143025,-0.000508333,0.0242225,-0.0311679,-0.0844807,-0.0527826,-0.134575,-0.300299,0.0807582,-0.103259,-0.0339295,0.194113,0.00302163,-0.232018,0.183362,0.125311,0.0559339,-0.200824,0.100802,-0.0625727,-0.0414394,-0.00649851,-0.271884,-0.0275973,0.0898061,-0.121293,-0.164997,0.116328,0.350312,0.0203967,-0.01532,-0.109209,0.0887381,-0.220377,0.0697567,-0.0801428,-0.0287647,0.130364,-0.0422939,0.0633204,0.00150056,-0.111577,0.0389992,0.0839094,0.0669141,-0.00381826,-0.0178501,-0.109518,-0.0203823,-0.14272,-0.0163341,0.0155258,-0.160649,-0.187713,0.0909331,-0.0117668,0.0958673,-0.251507,0.0977499,-0.235069,0.0751441,-0.121003,0.1066,-0.186996,0.038842,-0.125605,-0.0352568,-0.000846354,-0.0669478,0.0685601,-0.057111,-0.116316,0.0174296,-0.0338879,-0.0623175,0.104836,0.0677168,0.0499941,-0.125392,-0.00913227,0.203413,-0.167972,-0.00367533,-0.0581364,0.14796,-0.101187,0.30293,-0.113993,-0.0720578,0.044962,-0.0422213,-0.00148948
-0.0411143,-0.18323,-0.00488394,-0.0296727,0.0918286,-0.0578487,0.114483,0.214685,0.0612558,-0.0163542,0.148928,-0.0913887,-0.0615875,-0.126263,-0.118428,-0.203697,0.117475,-0.202226,-0.147388,0.157314,0.137634,-0.00876394,0.119352,0.085425,-0.00588222,-0.144601,0.127552,-0.120203,-0.0783861,0.0283912,-0.239652,-0.135632,0.0272464,-0.0706077,-0.0885793,-0.0518372,0.224037,0.0331913,0.0226432,0.0157093,-0.0232196,-0.0754991,0.0214566,-0.035631,-0.0790629,0.153112,-0.104327,0.0185604,-0.0164273,-0.0507387,0.0678177,0.0153213,-0.0770876,0.0486663,0.129321,-0.0376274,0.00304052,-0.124351,-0.0606938,-0.0049937,-0.110103,-0.111869,0.116424,0.0974101,0.0876954,-0.0703851,0.0544007,-0.121447,0.0308085,0.0684521,0.175338,-0.20544,-0.0393109,-0.089659,-0.0110223,-0.0345273,-0.162547,0.163878,-0.0286499,-0.186911,0.060524,-0.125388,0.0111062,0.0203173,-0.0335402,-0.0647333,-0.0586947,0.0440487,0.160874,-0.0911625,-0.0544053,-0.0495039,-0.0620164,0.0735328,0.237238,-0.0884225,0.0263291,-0.0189554,-0.120184,-0.0392095
nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
-0.0331148,-0.105623,-0.0899986,-0.04033,0.055419,-0.0524404,0.15729,0.253183,0.0952466,-0.00842362,0.0749748,-0.148339,0.00740833,-0.0497456,-0.137387,-0.180671,0.0890175,-0.170328,-0.0592206,0.151138,0.0851469,0.0124228,0.127153,0.0196362,-0.0121758,-0.22941,0.163483,-0.13434,-0.0410299,0.0216211,-0.152035,-0.0285557,0.118573,-0.0149605,-0.137879,-0.0228922,0.211693,-0.00790672,0.000488963,0.0277794,0.142604,-0.0391114,-0.0200254,-0.0111615,-0.0729647,0.249439,-0.0527152,0.0924203,0.0637349,-0.0227997,0.157045,-0.0620058,-0.0406295,0.0438578,0.152058,0.0289125,-0.0985556,-0.168036,-0.0938038,0.111626,-0.181619,-0.0667556,0.0773956,0.0607042,0.115258,-0.0787047,0.0650496,-0.12207,0.054857,0.00884414,0.186669,-0.244534,-0.0683264,-0.100868,-0.00288507,-0.0630574,-0.0627064,0.0890322,0.00534892,-0.149176,0.0198328,-0.158041,0.0519668,0.0284438,-0.0731035,0.0491306,-0.0463645,-0.0216871,0.199056,-0.0803353,-0.00477103,-0.118015,0.0462323,0.0126319,0.257169,-0.0618337,0.120326,0.0348068,-0.0132236,-0.0554333
nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na

In [123]:
# Add aggregated word embeddings 
loans = loans.cbind(desc_vecs)

# Step 5 (of 8). Model training

Now that we have cleaned our data and added new columns, we will train a model to predict bad loans.

In [124]:
train, test = loans.split_frame(seed = 1234, ratios = [0.75], destination_frames=["train.hex", "test.hex"])

In [125]:
from h2o.estimators import H2OGradientBoostingEstimator

cols_to_remove = ["initial_list_status",
                  "out_prncp",
                  "out_prncp_inv",
                  "total_pymnt",
                  "total_pymnt_inv",
                  "total_rec_prncp", 
                  "total_rec_int",
                  "total_rec_late_fee",
                  "recoveries",
                  "collection_recovery_fee",
                  "last_pymnt_d", 
                  "last_pymnt_amnt",
                  "next_pymnt_d",
                  "last_credit_pull_d",
                  "collections_12_mths_ex_med" , 
                  "mths_since_last_major_derog",
                  "policy_code",
                  "loan_status",
                  "funded_amnt",
                  "funded_amnt_inv",
                  "mths_since_last_delinq",
                  "mths_since_last_record",
                  "id",
                  "member_id",
                  "desc",
                  "zip_code"]

predictors = list(set(loans.col_names) - set(cols_to_remove))

In [126]:
predictors

['C25',
 'C11',
 'pymnt_plan',
 'C78',
 'C73',
 'dti',
 'C36',
 'C80',
 'C1',
 'inq_last_6mths',
 'C42',
 'loan_amnt',
 'title',
 'C60',
 'pub_rec',
 'C10',
 'annual_inc',
 'grade',
 'issue_d',
 'revol_bal',
 'C34',
 'open_acc',
 'credit_length',
 'C52',
 'C71',
 'C63',
 'C95',
 'installment',
 'C3',
 'C58',
 'C5',
 'term',
 'C40',
 'C91',
 'url',
 'total_acc',
 'C70',
 'C100',
 'C59',
 'C32',
 'C72',
 'int_rate',
 'C41',
 'C83',
 'C22',
 'C66',
 'C33',
 'C55',
 'C12',
 'C2',
 'C87',
 'C21',
 'C68',
 'sub_grade',
 'earliest_cr_line',
 'revol_util',
 'C86',
 'C49',
 'emp_title',
 'C38',
 'C69',
 'C9',
 'C37',
 'C90',
 'C57',
 'C76',
 'C16',
 'C93',
 'C31',
 'C74',
 'C44',
 'C97',
 'C8',
 'C46',
 'C20',
 'C77',
 'delinq_2yrs',
 'C75',
 'C85',
 'C4',
 'C98',
 'C88',
 'C14',
 'C6',
 'C50',
 'C84',
 'C26',
 'C28',
 'C94',
 'C45',
 'emp_length',
 'C19',
 'C82',
 'C15',
 'C27',
 'C39',
 'C56',
 'C96',
 'C48',
 'C29',
 'C24',
 'C13',
 'C67',
 'verification_status',
 'C47',
 'C79',
 'bad_loan',

In [127]:
gbm_model = H2OGradientBoostingEstimator(stopping_metric = "logloss",
                                         stopping_rounds = 5, # early stopping
                                         score_tree_interval = 5,
                                         ntrees = 500,
                                         model_id = "gbm.hex")
gbm_model.train(x = predictors,
                y = "bad_loan",
                training_frame = train,
                validation_frame = test)

gbm Model Build progress: |███████████████████████████████████████████████| 100%


The ROC curve of the training and testing data are shown below.  The area under the ROC curve is much higher for the training data than the testing data indicating that the model may be beginning to memorize the training data.

In [128]:
print("Training Data")
gbm_model.model_performance(train = True)
print("Testing Data")
gbm_model.model_performance(valid = True)

Training Data
Testing Data

ModelMetricsBinomial: gbm
** Reported on validation data. **

MSE: 0.1136424981640575
RMSE: 0.3371090300838254
LogLoss: 0.37228813959048623
Mean Per-Class Error: 0.2666032658181745
AUC: 0.8131550015850213
Gini: 0.6263100031700426
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.2683759655133148: 


,0,1,Error,Rate
0,6730.0,935.0,0.122,(935.0/7665.0)
1,873.0,1158.0,0.4298,(873.0/2031.0)
Total,7603.0,2093.0,0.1865,(1808.0/9696.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.2683760,0.5615907,207.0
max f2,0.1133028,0.6469274,313.0
max f0point5,0.5291741,0.6739090,114.0
max accuracy,0.5077639,0.8540635,120.0
max precision,0.9959165,1.0,0.0
max recall,0.0208756,1.0,397.0
max specificity,0.9959165,1.0,0.0
max absolute_mcc,0.5033637,0.4973999,121.0
max min_per_class_accuracy,0.1679737,0.7257509,267.0
max mean_per_class_accuracy,0.2102446,0.7333967,238.0


Gains/Lift Table: Avg response rate: 20.95 %, avg score: 20.78 %



,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.0100041,0.9750318,4.7740030,4.7740030,1.0,0.9833094,1.0,0.9833094,0.0477597,0.0477597,377.4002954,377.4002954
,2,0.0200083,0.9643698,4.7740030,4.7740030,1.0,0.9697594,1.0,0.9765344,0.0477597,0.0955194,377.4002954,377.4002954
,3,0.0300124,0.9509338,4.7247864,4.7575974,0.9896907,0.9580675,0.9965636,0.9703788,0.0472674,0.1427868,372.4786429,375.7597446
,4,0.0400165,0.9265712,4.4294873,4.6755699,0.9278351,0.9411332,0.9793814,0.9630674,0.0443131,0.1871000,342.9487277,367.5569904
,5,0.0500206,0.8830283,4.4294873,4.6263534,0.9278351,0.9087290,0.9690722,0.9521997,0.0443131,0.2314131,342.9487277,362.6353378
,6,0.1000413,0.4725776,3.1400143,3.8831838,0.6577320,0.6448125,0.8134021,0.7985061,0.1570655,0.3884786,214.0014314,288.3183846
,7,0.1500619,0.3514653,1.6438319,3.1367332,0.3443299,0.4047331,0.6570447,0.6672484,0.0822255,0.4707041,64.3831945,213.6733213
,8,0.2000825,0.2840819,1.4568091,2.7167522,0.3051546,0.3155105,0.5690722,0.5793140,0.0728705,0.5435746,45.6809149,171.6752197
,9,0.3000206,0.2021509,1.2316829,2.2220694,0.2579979,0.2383958,0.4654520,0.4657527,0.1230921,0.6666667,23.1682909,122.2069440
,10,0.4000619,0.1543394,0.8563675,1.8805559,0.1793814,0.1762389,0.3939160,0.3933556,0.0856721,0.7523387,-14.3632460,88.0555946


# Step 7 (of 8). Interpret model

In [129]:
loans["inq_last_6mths"].table().head(100)

inq_last_6mths,Count
0,17910
1,10288
2,5480
3,2942
4,1022
5,590
6,335
7,181
8,112
9,50


# Step 8 (of 8). Stop H2O-3 cluster

In [133]:
h2o.download_pojo(gbm_model,path="./",get_jar=True)

'/Users/sshkel/poc/k8s-experiments/gbm_hex.java'

In [43]:
h2o.cluster().shutdown()

H2O session _sid_8fbf closed.


# Bonus: Github location for this tutorial

* https://github.com/h2oai/h2o-tutorials/tree/master/nyc-workshop-2018/h2o_sw/h2o-3-hands-on

# Bonus: H2O-3 documentation

* http://docs.h2o.ai